## Regression model training

* open the sample of training points created in the preprocessing notebook
* sample the predictors and the LST (10 m) at those locations

In [1]:
import geopandas as gpd
import rasterio
from rasterstats import point_query
from shapely.geometry import mapping
import pandas as pd
import math
import os
import numpy as np
import ipywidgets as widgets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import ensemble
import seaborn as sns
import plotly.graph_objects as go
import sys
from rasterio.crs import CRS
import glob

#### Option 1: manual image selection

In [ ]:
total_samples = 5000

# date_s2 = '2023-06-25' #Summer
# season = "Summer"

# date_s2 = '2023-03-22' #Winter
# season = "Winter"

date_s2 = '2023-11-17' #Intermediate
season = "Intermediate"

#### Option 2: Tagged cell as parameters and selection of a random parameters that will be rewriten by and extarnal file
use this method if you intend to run with more than a single date

In [ ]:
# The following parameters are just and example and will be rewritten by driver.py
date_s2 = '2023-11-17' #Intermediate
season = "Intermediate"
total_samples = 5000

### Sample predictors and LST at the point locations and save the values in a dataframe

In [ ]:
# Path pattern for seasonal GeoPackages
file_pattern = f"/{season}_sampled_points_*_{total_samples}_all_bands.gpkg"

# Find all matching files
file_list = glob.glob(file_pattern)

# Load and combine all GeoPackages
gdf_list = []
for file in file_list:
    print(f"Loading: {file}")
    gdf = gpd.read_file(file)
    
    # Optional: Add date as a new column (parsed from filename)
    date_str = os.path.basename(file).split('_')[3]  # Extract yyyymmdd
    gdf["date"] = pd.to_datetime(date_str, format='%Y%m%d')
    
    gdf_list.append(gdf)

# Concatenate into one GeoDataFrame
df_full = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)

print(f"\nTotal stacked samples: {len(df_full)}")

In [46]:
summary = df_full.describe()

In [ ]:
df_full

# Modelling
* split the points into training and testing samples
* perform cross-validation for hyperparameter tuning
* after the selection of the best parameters, evaluate the model error (RMSE, MAE) and coefficient of determination (R2)
* use the model to predict LST in all pixels of the AOI

In [ ]:
df_X = df_full.drop(columns=["LCZ", "LST", "geometry", "date"])
print(df_X)
series_y = df_full.iloc[:, 1]
print(series_y)

### 1. Split the points into training and testing samples

In [50]:
X_train, X_test, y_train, y_test = train_test_split(df_X, series_y, train_size = 0.7, random_state = 42)

In [ ]:
# Print dataset sizes
print(f"Train size: {X_train.shape}, {y_train.shape}")
print(f"Test size: {X_test.shape}, {y_test.shape}")

### 2. K-fold cross-validation

In [52]:
param_grid = {
    'n_estimators': [10, 20, 50, 100, 200],
    'max_depth': [5, 10, 20, 30, 50],
    'max_features': [1.0, 'sqrt'],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

In [ ]:
# Select best hyperparameters
scoring_metric = 'neg_mean_squared_error'
#scoring_metric = 'r2'
grid_search = GridSearchCV(ensemble.RandomForestRegressor(), param_grid, scoring = scoring_metric, cv = 5, verbose = 1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(f"Best hyperparameters: ", best_params)

In [ ]:
# Extract the negative mean test scores
scores = -grid_search.cv_results_['mean_test_score']

# Create the histogram
plt.figure(figsize=(6, 4))
plt.hist(np.sqrt(scores), bins=30, color='orange', edgecolor='black', alpha=0.75)

# Update layout
plt.title(f'Distribution of Cross-Validation Scores for Season: {season}')
plt.xlabel('Average RMSE (°K) in the folds')
plt.ylabel('Count')
plt.grid(False)

# Show plot
plt.tight_layout()
plt.show()

### 3. Build the model and evaluate it on the training and testing sets

In [ ]:
regr = ensemble.RandomForestRegressor(n_estimators = best_params['n_estimators'],
                                     max_depth = best_params['max_depth'],
                                     max_features = best_params['max_features'],
                                     min_samples_split = best_params['min_samples_split'],
                                     min_samples_leaf = best_params['min_samples_leaf'])

regr.fit(X_train, y_train)

In [ ]:
import os
import pickle

# Create output directory with model name
model_folder = f"/saved_model_{season}_{total_samples}_samples"
os.makedirs(model_folder, exist_ok=True)

# Save best_params
with open(os.path.join(model_folder, "best_params.pkl"), "wb") as f:
    pickle.dump(best_params, f)

# Save trained model
with open(os.path.join(model_folder, "regr_model.pkl"), "wb") as f:
    pickle.dump(regr, f)

# Save sampled data (df_full)
df_full.to_csv(os.path.join(model_folder, "sampled_data.csv"), index=False)

print(f"Saved model and parameters to: {model_folder}")

In [57]:
y_pred_test = regr.predict(X_test)
y_pred_train = regr.predict(X_train)

In [ ]:
mae = mean_absolute_error(y_train, y_pred_train)
print("MAE on TRAINING: ", round(mae, 3))
mae = mean_absolute_error(y_test, y_pred_test)
print("MAE on TESTING: ", round(mae, 3))

In [ ]:
rmse = mean_squared_error(y_train, y_pred_train)
rmse = math.sqrt(rmse)
print("RMSE on TRAINING: ", round(rmse, 3))
rmse = mean_squared_error(y_test, y_pred_test)
rmse = math.sqrt(rmse)
print("RMSE on TESTING: ", round(rmse, 3))

In [ ]:
r2 = r2_score(y_train, y_pred_train)
print("R2 on TRAINING: ", round(r2, 3))
r2 = r2_score(y_test, y_pred_test)
print("R2 on TESTING: ", round(r2, 3))

In [ ]:
# Mean Bias Error (MBE)
mbe_train = np.mean(y_pred_train - y_train)
mbe_test = np.mean(y_pred_test - y_test)

print("MBE on TRAINING: ", round(mbe_train, 3))
print("MBE on TESTING: ", round(mbe_test, 3))

# Mean Absolute Bias (same as MAE, already computed earlier)
mab_train = np.mean(np.abs(y_pred_train - y_train))
mab_test = np.mean(np.abs(y_pred_test - y_test))

print("MAB on TRAINING: ", round(mab_train, 3))
print("MAB on TESTING: ", round(mab_test, 3))

### 4. Evaluate feature importance

In [64]:
feature_importances = regr.feature_importances_
features = X_train.columns

# Create a DataFrame for better visualization
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importances
})

# Sort by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

In [ ]:
# Create the bar chart
plt.figure(figsize=(6, 4))
plt.barh(
    importance_df['Feature'],
    importance_df['Importance'],
    color='skyblue',
    edgecolor='black'
)

# Add titles and labels
plt.title(f'Feature Importances for Season: {season}')
plt.xlabel('Importance')
plt.ylabel('Feature')

# Tight layout and show plot
plt.tight_layout()
plt.show()

### 5. Stack the predictors and put them in a dataframe

In [ ]:
file_l8 = f"Average_LST_map_{season}.tif"
# === Step 2: List of raster files to sample ===
rasters = {
    'LST': '\Input\LST_L89\Mediated_LST/' + file_l8[:-4] + '_10m.tif',
    'BH': '\UCP/UCP_20m/BH_10m.tif',
    'BSF': '\UCP/UCP_20m/BSF_10m.tif',
    'IMD': '\UCP/UCP_20m/IMD_10m.tif',
    'SVF': '\UCP/UCP_20m/SVF_10m.tif',
    'TCH': '\UCP/UCP_20m/TCH_10m.tif',
    'Fractions': ('Fractions/Final_Class_Fraction_Layer_Masked_' + date_s2 + '_10m.tif', 
                 ['F_S', 'F_M', 'F_AC', 'F_BS', 'F_TV', 'F_W', 'F_G'])
}

In [67]:
data = {}
reference_shape = None

# Helper function to validate shape
def check_shape(shape, name):
    global reference_shape
    if reference_shape is None:
        reference_shape = shape
    elif shape != reference_shape:
        raise ValueError(f"Shape mismatch for {name}: {shape} != {reference_shape}")

# Process rasters
for key, value in rasters.items():
    if key != 'Fractions':
        with rasterio.open(value) as src:
            arr = src.read(1)
            predictor_meta = src.meta
            check_shape(arr.shape, key)
            data[key] = arr.flatten()
    else:
        path, band_names = value
        with rasterio.open(path) as src:
            for i, band_name in enumerate(band_names, start=1):
                arr = src.read(i)
                check_shape(arr.shape, band_name)
                data['Fractions_' + band_name] = arr.flatten()

# Stack into DataFrame
df = pd.DataFrame(data)

### 6. Predict on all the pixels and export the prediction to TIF

In [ ]:
prediction = regr.predict(df.iloc[:, 1:])

In [ ]:
reference_shape

In [ ]:
prediction_reshaped = prediction.reshape(reference_shape[0], reference_shape[1])

In [ ]:
output_meta = predictor_meta.copy()

In [ ]:
with rasterio.open('\LST_L89/model_predictions/predicted_LST_' + date_s2.replace('-', '') + '_20m_' + str(total_samples) + '_samples.tif', 'w', **output_meta) as dst:
    dst.write(prediction_reshaped, 1)

In [ ]:
plt.figure(figsize=(12, 8))
plt.imshow(prediction_reshaped, cmap='Spectral_r')
#plt.title(f'Predicted Temperature for Date: {date_s2}')
plt.title(f'Predicted Temperature')
plt.colorbar()
plt.show()

### 7. Compute R<sup>2</sup>, RMSE, MAE and residuals

In [ ]:
fig = go.Figure()
# Combine all values to find global min and max
all_values = np.concatenate([y_train, y_pred_train, y_test, y_pred_test])
min_val = np.min(all_values)
max_val = np.max(all_values)

# Optional: pad the range slightly for visual clarity
padding = 2  # degrees K
min_val -= padding
max_val += padding
# Reference line: y = x (perfect prediction)
fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    line=dict(color='grey'),
    name='y = x'
))

# Scatter plot for training data
fig.add_trace(go.Scatter(
    x=y_train,
    y=y_pred_train,
    mode='markers',
    marker=dict(color='blue', opacity=0.5),
    name='Train'
))

# Scatter plot for test data
fig.add_trace(go.Scatter(
    x=y_test,
    y=y_pred_test,
    mode='markers',
    marker=dict(color='red', opacity=0.5),
    name='Test'
))

# Combine all values to find global min and max
all_values = np.concatenate([y_train, y_pred_train, y_test, y_pred_test])
min_val = np.min(all_values) - 2
max_val = np.max(all_values) + 2

# Update layout
fig.update_layout(
    title=f'Scatter Plot for Season: {season}',
    xaxis=dict(
        title='Regression Prediction (°K)',
        range=[min_val, max_val]
    ),
    yaxis=dict(
        title='True Value (°K)',
        range=[min_val, max_val],
        scaleanchor='x',
        scaleratio=1
    ),
    width=600,
    height=600,
    showlegend=True,
    template='simple_white'
)

fig.show()

### 8. Compute residuals (both for train and test) and export to Geopackage

In [ ]:
from sklearn.metrics import mean_absolute_error

# Compute residuals
residuals_train = y_train - y_pred_train
residuals_test = y_test - y_pred_test

# Add predictions and residuals back into the original DataFrame
train_df = X_train.copy()
train_df["Observed"] = y_train
train_df["Predicted"] = y_pred_train
train_df["Residual"] = residuals_train
train_df["Split"] = "train"

test_df = X_test.copy()
test_df["Observed"] = y_test
test_df["Predicted"] = y_pred_test
test_df["Residual"] = residuals_test
test_df["Split"] = "test"

# Combine train and test
combined_df = pd.concat([train_df, test_df])

In [ ]:
# Add index to align later
gdf = gdf.reset_index(drop=True)

# Ensure index is aligned with df_X
df_X = df_X.reset_index(drop=True)

# Join residual info to geometry
df_with_geometry = pd.concat([df_X, gdf[["geometry"]]], axis=1)

# Now join predictions & residuals back to spatial points
final_df = df_with_geometry.merge(combined_df[["Observed", "Predicted", "Residual", "Split"]], 
                                  left_index=True, right_index=True)

In [ ]:
# Convert to GeoDataFrame
final_gdf = gpd.GeoDataFrame(final_df, geometry="geometry", crs=gdf.crs)

# Create output folder if it doesn't exist
output_path = f"/model_predictions/residuals_{date_s2.replace('-', '')}.gpkg"
final_gdf.to_file(output_path, layer="residuals", driver="GPKG")

print(f"Residuals exported to {output_path}")

### 9. Compute the residuals on ALL pixels and export them as a TIF (map of differences between actual LST and predicted LST)

In [ ]:
import rasterio
from rasterio.features import rasterize
import numpy as np
import geopandas as gpd
import pandas as pd

# Step 1: Prepare your data.
crs = final_gdf.crs

# Base raster reference parameters (adjust these values to your actual base raster)
pixel_size = 10  # Example pixel size (adjust according to your dataset)
minx, miny, maxx, maxy = final_gdf.total_bounds
width = int((maxx - minx) / pixel_size)
height = int((maxy - miny) / pixel_size)
transform = rasterio.transform.from_origin(minx, maxy, pixel_size, pixel_size)

# Step 2: Prepare data for rasterization (geometry, value pairs)
shapes = ((geom, value) for geom, value in zip(final_gdf.geometry, final_gdf.Residual))

# Step 3: Rasterize the residuals
residual_raster = rasterize(
    shapes,
    out_shape=(height, width),
    fill=np.nan,  # No data value
    transform=transform,
    dtype='float32',
)

# Step 4: Export the raster to GeoTIFF
output_tif_path = f'/residuals_20m_' + str(total_samples) + '_samples.tif'

with rasterio.open(
    output_tif_path,
    'w',
    driver='GTiff',
    height=residual_raster.shape[0],
    width=residual_raster.shape[1],
    count=1,
    dtype=residual_raster.dtype,
    crs=crs,
    transform=transform,
    nodata=np.nan
) as dst:
    dst.write(residual_raster, 1)

print(f"Residuals raster exported to {output_tif_path}")

# Apply Model on Data with changed UCPs and Fractions

In [75]:
from rasterio.warp import reproject, Resampling, calculate_default_transform
from rasterio.enums import Resampling

In [ ]:
lst_file_path = os.path.join('\LST_L89\Mediated_LST/', file_l8)

In [ ]:
BH_file_path = '\simulated_covariates/BH_pct.tif'
BSF_file_path = '\simulated_covariates/BSF_pct.tif'
IMD_file_path = '\simulated_covariates/IMD_pct.tif'
SVF_file_path = '\simulated_covariates/SVF_pct.tif'
TCH_file_path = '\simulated_covariates/TCH_pct.tif'

In [ ]:
fractions = '\Final_Class_Fraction_Layer_Masked_' + date_s2 + '.tif'
fractions = '\simulated_covariates/Simulated_Fraction_1.tif'


In [79]:
input_predictors = [
    lst_file_path,
    BH_file_path,
    BSF_file_path,
    IMD_file_path,
    SVF_file_path,
    TCH_file_path,
    fractions
]

In [ ]:
for input_path in input_predictors:
    
    output_path = input_path[:-4] + '_10m.tif'
    
    if not os.path.exists(output_path):
        with rasterio.open(input_path) as src:
            src_crs = src.crs or CRS.from_epsg(32632)
            dst_resolution = 10

            transform, width, height = calculate_default_transform(
                src_crs, src_crs, src.width, src.height, *src.bounds, resolution=dst_resolution
            )

            kwargs = src.meta.copy()
            kwargs.update({
                'transform': transform,
                'width': width,
                'height': height,
                'res': (dst_resolution, dst_resolution),
                'compress': 'lzw'
            })

            with rasterio.open(output_path, 'w', **kwargs) as dst:
                
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.nearest
                    )
        print(f"Processed: {output_path}")
    else:
        print(f"Skipped (already exists): {output_path}")

In [81]:
reference = fractions[:-4] + '_10m.tif'

In [82]:
input_rasters = [
    BH_file_path[:-4] + '_10m.tif',
    BSF_file_path[:-4] + '_10m.tif',
    IMD_file_path[:-4] + '_10m.tif',
    SVF_file_path[:-4] + '_10m.tif',
    TCH_file_path[:-4] + '_10m.tif',
    lst_file_path[:-4] + '_10m.tif'
]

In [ ]:
with rasterio.open(reference) as ref:
    ref_crs = ref.crs or CRS.from_epsg(32632)
    ref_transform = ref.transform
    ref_width = ref.width
    ref_height = ref.height

for raster_path in input_rasters:
    with rasterio.open(raster_path) as src:
        profile = src.profile
        profile.update({
            'crs': ref_crs,
            'transform': ref_transform,
            'width': ref_width,
            'height': ref_height
        })

        # Create an empty array to store the aligned data
        aligned = np.empty((ref_height, ref_width), dtype=src.dtypes[0])

        # Reproject and resample from source to aligned array
        reproject(
            source=src.read(1),  # Read band 1
            destination=aligned,
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=ref_transform,
            dst_crs=ref_crs,
            resampling=Resampling.nearest
        )

    # Overwrite the original file with the aligned version
    with rasterio.open(raster_path, 'w', **profile) as dst:
        dst.write(aligned, 1)

    print(f'Aligned and overwritten: {raster_path}')

In [ ]:
file_l8 = f"Average_LST_map_{season}.tif"
# === Step 2: List of raster files to sample ===
rasters_pct = {
    'LST': '\Input\LST_L89\Mediated_LST/' + file_l8[:-4] + '_10m.tif',
    'BH': '\simulated_covariates/BH_pct_10m.tif',
    'BSF': '\simulated_covariates/BSF_pct_10m.tif',
    'IMD': '\simulated_covariates/IMD_pct_10m.tif',
    'SVF': '\simulated_covariates/SVF_pct_10m.tif',
    'TCH': '\simulated_covariates/TCH_pct_10m.tif',
    'Fractions': ('simulated_covariates\Simulated_Fraction_1_10m.tif', 
                 ['F_S', 'F_M', 'F_AC', 'F_BS', 'F_TV', 'F_W', 'F_G'])
}

In [88]:
data = {}
reference_shape = None

# Helper function to validate shape
def check_shape(shape, name):
    global reference_shape
    if reference_shape is None:
        reference_shape = shape
    elif shape != reference_shape:
        raise ValueError(f"Shape mismatch for {name}: {shape} != {reference_shape}")

# Process rasters
for key, value in rasters_pct.items():
    if key != 'Fractions':
        with rasterio.open(value) as src:
            arr = src.read(1)
            predictor_meta = src.meta
            check_shape(arr.shape, key)
            data[key] = arr.flatten()
    else:
        path, band_names = value
        with rasterio.open(path) as src:
            for i, band_name in enumerate(band_names, start=1):
                arr = src.read(i)
                check_shape(arr.shape, band_name)
                data['Fractions_' + band_name] = arr.flatten()

# Stack into DataFrame
df = pd.DataFrame(data)

In [89]:
prediction = regr.predict(df.iloc[:, 1:])

In [90]:
prediction_reshaped = prediction.reshape(reference_shape[0], reference_shape[1])

In [ ]:
output_meta = predictor_meta.copy()
with rasterio.open('/simulated_covariates_predicted_LST_' + date_s2.replace('-', '') + '_20m_' + str(total_samples) + '_samples.tif', 'w', **output_meta) as dst:
    dst.write(prediction_reshaped, 1)

In [ ]:
plt.figure(figsize=(12, 8))
plt.imshow(prediction_reshaped, cmap='Spectral_r')
plt.title(f'Predicted Temperature')
plt.colorbar()
plt.show()

In [ ]:
fig = go.Figure()

# Reference line: y = x (perfect prediction)
fig.add_trace(go.Scatter(
    x=y_train,
    y=y_train,
    mode='lines',
    line=dict(color='grey'),
    name='y = x'
))

# Scatter plot for training data
fig.add_trace(go.Scatter(
    x=y_train,
    y=y_pred_train,
    mode='markers',
    marker=dict(color='blue', opacity=0.5),
    name='Train'
))

# Scatter plot for test data
fig.add_trace(go.Scatter(
    x=y_test,
    y=y_pred_test,
    mode='markers',
    marker=dict(color='red', opacity=0.5),
    name='Test'
))

fig.update_layout(
    title=f'Scatter Plot for Season: {season}',
    xaxis_title='Regression Prediction (°K)',
    yaxis_title='True Value (°K)',
    width=600,
    height=600,
    showlegend=True,
    template='simple_white'
)

fig.show()